# Project Geminae MidPoint Model
## Gradient Boosted Regression Model for 3 and 6 month projections

Tom Gregg

2024-02-25

## Setting Up The Model

In [1]:
# Import Basic Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
from datetime import datetime

In [2]:
# Importing Libraries and Packages to perform Boosted Tree
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from xgboost import XGBRegressor

In [3]:
# Max Display 
pd.options.display.max_columns = None
pd.options.display.max_rows = None

## Importing and Preparing Data

In [4]:
# Creating our file path for the CSV
file_path = 'https://raw.githubusercontent.com/tbgregg000/Capstone/main/GenericWellDataPrepped.csv'
df = pd.read_csv(file_path).copy()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16707 entries, 0 to 16706
Data columns (total 89 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Well Index                               16707 non-null  int64  
 1   TrueVerticalDepth_FT                     16707 non-null  float64
 2   MeasuredDepth_FT                         16707 non-null  float64
 3   UpperPerforation_FT                      16707 non-null  float64
 4   LowerPerforation_FT                      16707 non-null  float64
 5   PerforationInterval_FT                   16707 non-null  float64
 6   LateralLength_FT                         16707 non-null  float64
 7   ProppantLoad_LBSPerGAL                   16707 non-null  float64
 8   ProppantIntensity_LBSPerFT               16707 non-null  float64
 9   TotalProppant_LBS                        16707 non-null  float64
 10  TotalWaterPumped_GAL                     16707

In [6]:
# Dropping 2020 since the data in this year is thrown off
df = df[df['YearOfDrilling'] != 2020]
# df = df[df['YearOfDrilling'] >= 2017]
# Drop any null 12 month values


In [7]:
df.dropna(subset=['First12MonthGas_MCFPer1000FT'], inplace=True)

In [8]:
# df['YearOfDrilling'].value_counts()

In [9]:
df_cleaned = df.copy()

In [10]:
# Splitting data into Water, Gas, and Oil 
# Splitting data into 3 month and 6 month
y_w_3 = df_cleaned['First3MonthWater_BBL']
y_g_3 = df_cleaned['First3MonthGas_MCF']
y_o_3 = df_cleaned['First3MonthOil_BBL']
y_w_6 = df_cleaned['First6MonthWater_BBL']
y_g_6 = df_cleaned['First6MonthGas_MCF']
y_o_6 = df_cleaned['First6MonthOil_BBL']
y_w_9 = df_cleaned['First9MonthWater_BBL']
y_g_9 = df_cleaned['First9MonthGas_MCF']
y_o_9 = df_cleaned['First9MonthOil_BBL']
y_w_12 = df_cleaned['First12MonthWater_BBL']
y_g_12 = df_cleaned['First12MonthGas_MCF']
y_o_12 = df_cleaned['First12MonthOil_BBL']
# y_w_36 = df_cleaned['First36MonthWater_BBL']
# y_g_36 = df_cleaned['First36MonthGas_MCFPer1000FT']
# y_o_36 = df_cleaned['First36MonthOil_BBLPer1000FT']
y_w_peak = df_cleaned['PeakWater_BBL']
y_g_peak = df_cleaned['PeakGas_MCF']
y_o_peak = df_cleaned['PeakOil_BBL']
y_w_cum = df_cleaned['CumWater_BBL']
y_g_cum = df_cleaned['CumGas_MCF']
y_o_cum = df_cleaned['CumOil_BBL']

In [11]:
# Creating X using just the non-production columns
X = df_cleaned.iloc[:, :26]
X = X.drop("Well Index", axis=1)

# Date Cleanup
columns_to_change = ['InitialProductionDate','DrillingStartDate','DrillingCompletionDate']
for col in columns_to_change:
    X[col] = pd.to_datetime(X[col])

# Loop through specific columns and rename
for col in columns_to_change:
    new_name = col + 'Num'
    X.rename(columns={col: new_name}, inplace=True)
    X[new_name] = X[new_name].astype('int64') / 10**9


# Dropping a few unnecessary columns
# X = X.drop('InitialProductionMonth', axis = 1)
X = X.drop('DrillingCompletionDateNum', axis = 1)
X = X.drop('DrillingDuration_DAYS', axis = 1)
# X = X.drop('ProductionMonthsCount', axis = 1)
X = X.drop('YearOfDrilling', axis = 1)
X = X.drop('InitialProductionYear', axis = 1)


# # Dummy Variables for OilTest_Method
# # Use pd.get_dummies to create dummy variables
# dummy_vars = pd.get_dummies(X['OilTest_Method'], prefix='OilTest_Method', drop_first=True)

# # Add the dummy variables as new columns to your DataFrame
# X = pd.concat([X.drop("OilTest_Method", axis=1), dummy_vars], axis=1)

# Converting Objects to Ints
# for col in X.columns:
#     if pd.api.types.is_object_dtype(X[col]):
#         X[col] = X[col].str.replace(',', '')
#         X[col] = X[col].str.replace(' ', '')
#         X[col] = X[col].astype(float)

In [12]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13940 entries, 0 to 16140
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   TrueVerticalDepth_FT        13940 non-null  float64
 1   MeasuredDepth_FT            13940 non-null  float64
 2   UpperPerforation_FT         13940 non-null  float64
 3   LowerPerforation_FT         13940 non-null  float64
 4   PerforationInterval_FT      13940 non-null  float64
 5   LateralLength_FT            13940 non-null  float64
 6   ProppantLoad_LBSPerGAL      13940 non-null  float64
 7   ProppantIntensity_LBSPerFT  13940 non-null  float64
 8   TotalProppant_LBS           13940 non-null  float64
 9   TotalWaterPumped_GAL        13940 non-null  float64
 10  WaterIntensity_GALPerFT     13940 non-null  float64
 11  TotalFluidPumped_BBL        13940 non-null  float64
 12  FluidIntensity_BBLPerFT     13940 non-null  float64
 13  AcidVolume_BBL              139

In [13]:
X.head()

,TrueVerticalDepth_FT,MeasuredDepth_FT,UpperPerforation_FT,LowerPerforation_FT,PerforationInterval_FT,LateralLength_FT,ProppantLoad_LBSPerGAL,ProppantIntensity_LBSPerFT,TotalProppant_LBS,TotalWaterPumped_GAL,WaterIntensity_GALPerFT,TotalFluidPumped_BBL,FluidIntensity_BBLPerFT,AcidVolume_BBL,OilTest_Method_FLOWING,OilTest_Method_GAS LIFT,OilTest_Method_PUMPING,FractureStages,AvgStageSpacing_FT,InitialProductionDateNum,DrillingStartDateNum
0,11890.0,15995.0,11878.0,15811.0,3933.0,3718.0,4.445714,1802.428571,1.429378e+07,404320.0,103.000000,9627.000000,2.000000,1216.428571,0.0,0.0,1.0,NaN,NaN,1.296518e+09,1.141430e+09
1,9141.0,13493.0,9470.0,13400.0,3930.0,4076.0,0.770000,672.000000,2.641750e+06,3412481.0,868.000000,81250.000000,21.000000,555.142857,0.0,0.0,1.0,NaN,NaN,1.296518e+09,1.290125e+09
2,9688.0,13824.0,10082.0,13739.0,3657.0,3943.0,0.990000,1094.000000,4.000020e+06,4051150.0,1108.000000,96456.000000,26.000000,708.857143,0.0,0.0,1.0,NaN,NaN,1.298938e+09,1.284336e+09
3,9187.0,13542.0,9451.0,13436.0,3985.0,4145.0,0.470000,560.000000,2.233000e+06,4768999.0,1197.000000,113548.000000,28.000000,675.428571,0.0,1.0,0.0,NaN,NaN,1.298938e+09,1.294099e+09
4,7935.0,12859.0,8478.0,12690.0,4212.0,4564.0,0.841429,1195.000000,5.033010e+06,6029816.0,1420.714286,145108.285714,34.142857,716.857143,0.0,0.0,1.0,15.0,304.0,1.301616e+09,1.295222e+09


In [53]:
# Creating the test and train split using seed 99
# Quite nice how we can just use the exact same X set

# X_train, X_test, y_train_w_3, y_test_w_3 = train_test_split(X, y_w_3, test_size=0.2, random_state=965)

X_train, X_rest1, y_train_w_3, y_rest_w_3_1 = train_test_split(X, y_w_3, test_size=2000, random_state=965)
X_test, X_rest2, y_test_w_3, y_rest_w_3_2 = train_test_split(X_rest1, y_rest_w_3_1, test_size = 1500, random_state=965)
X_calib, X_new, y_calib_w_3, y_new_w_3 = train_test_split(X_rest2, y_rest_w_3_2, test_size = 500, random_state=965)

# X_train, X_test, y_train_g_3, y_test_g_3 = train_test_split(X, y_g_3, test_size=0.2, random_state=965)
X_train, X_rest1, y_train_g_3, y_rest_g_3_1 = train_test_split(X,y_g_3, test_size=2000, random_state=965)
X_test, X_rest2, y_test_g_3, y_rest_g_3_2 = train_test_split(X_rest1, y_rest_g_3_1, test_size = 1500, random_state=965)
X_calib, X_new, y_calib_g_3, y_new_g_3 = train_test_split(X_rest2, y_rest_g_3_2, test_size = 500, random_state=965)

# X_train, X_test, y_train_o_3, y_test_o_3 = train_test_split(X, y_o_3, test_size=0.2, random_state=965)
X_train, X_rest1, y_train_o_3, y_rest_o_3_1 = train_test_split(X,y_o_3, test_size=2000, random_state=965)
X_test, X_rest2, y_test_o_3, y_rest_o_3_2 = train_test_split(X_rest1, y_rest_o_3_1, test_size = 1500, random_state=965)
X_calib, X_new, y_calib_o_3, y_new_o_3 = train_test_split(X_rest2, y_rest_o_3_2, test_size = 500, random_state=965)

# X_train, X_test, y_train_w_6, y_test_w_6 = train_test_split(X, y_w_6, test_size=0.2, random_state=965)
X_train, X_rest1, y_train_w_6, y_rest_w_6_1 = train_test_split(X,y_w_6, test_size=2000, random_state=965)
X_test, X_rest2, y_test_w_6, y_rest_w_6_2 = train_test_split(X_rest1, y_rest_w_6_1, test_size = 1500, random_state=965)
X_calib, X_new, y_calib_w_6, y_new_w_6 = train_test_split(X_rest2, y_rest_w_6_2, test_size = 500, random_state=965)

# X_train, X_test, y_train_g_6, y_test_g_6 = train_test_split(X, y_g_6, test_size=0.2, random_state=965)
X_train, X_rest1, y_train_g_6, y_rest_g_6_1 = train_test_split(X,y_g_6, test_size=2000, random_state=965)
X_test, X_rest2, y_test_g_6, y_rest_g_6_2 = train_test_split(X_rest1, y_rest_g_6_1, test_size = 1500, random_state=965)
X_calib, X_new, y_calib_g_6, y_new_g_6 = train_test_split(X_rest2, y_rest_g_6_2, test_size = 500, random_state=965)

# X_train, X_test, y_train_o_6, y_test_o_6 = train_test_split(X, y_o_6, test_size=0.2, random_state=965)
X_train, X_rest1, y_train_o_6, y_rest_o_6_1 = train_test_split(X,y_o_6, test_size=2000, random_state=965)
X_test, X_rest2, y_test_o_6, y_rest_o_6_2 = train_test_split(X_rest1, y_rest_o_6_1, test_size = 1500, random_state=965)
X_calib, X_new, y_calib_o_6, y_new_o_6 = train_test_split(X_rest2, y_rest_o_6_2, test_size = 500, random_state=965)

# X_train, X_test, y_train_w_9, y_test_w_9 = train_test_split(X, y_w_9, test_size=0.2, random_state=965)
X_train, X_rest1, y_train_w_9, y_rest_w_9_1 = train_test_split(X,y_w_9, test_size=2000, random_state=965)
X_test, X_rest2, y_test_w_9, y_rest_w_9_2 = train_test_split(X_rest1, y_rest_w_9_1, test_size = 1500, random_state=965)
X_calib, X_new, y_calib_w_9, y_new_w_9 = train_test_split(X_rest2, y_rest_w_9_2, test_size = 500, random_state=965)

# X_train, X_test, y_train_g_9, y_test_g_9 = train_test_split(X, y_g_9, test_size=0.2, random_state=965)
X_train, X_rest1, y_train_g_9, y_rest_g_9_1 = train_test_split(X,y_g_9, test_size=2000, random_state=965)
X_test, X_rest2, y_test_g_9, y_rest_g_9_2 = train_test_split(X_rest1, y_rest_g_9_1, test_size = 1500, random_state=965)
X_calib, X_new, y_calib_g_9, y_new_g_9 = train_test_split(X_rest2, y_rest_g_9_2, test_size = 500, random_state=965)

# X_train, X_test, y_train_o_9, y_test_o_9 = train_test_split(X, y_o_9, test_size=0.2, random_state=965)
X_train, X_rest1, y_train_o_9, y_rest_o_9_1 = train_test_split(X,y_o_9, test_size=2000, random_state=965)
X_test, X_rest2, y_test_o_9, y_rest_o_9_2 = train_test_split(X_rest1, y_rest_o_9_1, test_size = 1500, random_state=965)
X_calib, X_new, y_calib_o_9, y_new_o_9 = train_test_split(X_rest2, y_rest_o_9_2, test_size = 500, random_state=965)

# X_train, X_test, y_train_w_12, y_test_w_12 = train_test_split(X, y_w_12, test_size=0.2, random_state=965)
X_train, X_rest1, y_train_w_12, y_rest_w_12_1 = train_test_split(X,y_w_12, test_size=2000, random_state=965)
X_test, X_rest2, y_test_w_12, y_rest_w_12_2 = train_test_split(X_rest1, y_rest_w_12_1, test_size = 1500, random_state=965)
X_calib, X_new, y_calib_w_12, y_new_w_12 = train_test_split(X_rest2, y_rest_w_12_2, test_size = 500, random_state=965)

# X_train, X_test, y_train_g_12, y_test_g_12 = train_test_split(X, y_g_12, test_size=0.2, random_state=965)
X_train, X_rest1, y_train_g_12, y_rest_g_12_1 = train_test_split(X,y_g_12, test_size=2000, random_state=965)
X_test, X_rest2, y_test_g_12, y_rest_g_12_2 = train_test_split(X_rest1, y_rest_g_12_1, test_size = 1500, random_state=965)
X_calib, X_new, y_calib_g_12, y_new_g_12 = train_test_split(X_rest2, y_rest_g_12_2, test_size = 500, random_state=965)

# X_train, X_test, y_train_o_12, y_test_o_12 = train_test_split(X, y_o_12, test_size=0.2, random_state=965)
X_train, X_rest1, y_train_o_12, y_rest_o_12_1 = train_test_split(X,y_o_12, test_size=2000, random_state=965)
X_test, X_rest2, y_test_o_12, y_rest_o_12_2 = train_test_split(X_rest1, y_rest_o_12_1, test_size = 1500, random_state=965)
X_calib, X_new, y_calib_o_12, y_new_o_12 = train_test_split(X_rest2, y_rest_o_12_2, test_size = 500, random_state=965)

# X_train, X_test, y_train_w_peak, y_test_w_peak = train_test_split(X, y_w_peak, test_size=0.2, random_state=965)
X_train, X_rest1, y_train_w_peak, y_rest_w_peak_1 = train_test_split(X,y_w_peak, test_size=2000, random_state=965)
X_test, X_rest2, y_test_w_peak, y_rest_w_peak_2 = train_test_split(X_rest1, y_rest_w_peak_1, test_size = 1500, random_state=965)
X_calib, X_new, y_calib_w_peak, y_new_w_peak = train_test_split(X_rest2, y_rest_w_peak_2, test_size = 500, random_state=965)

# X_train, X_test, y_train_g_peak, y_test_g_peak = train_test_split(X, y_g_peak, test_size=0.2, random_state=965)
X_train, X_rest1, y_train_g_peak, y_rest_g_peak_1 = train_test_split(X,y_g_peak, test_size=2000, random_state=965)
X_test, X_rest2, y_test_g_peak, y_rest_g_peak_2 = train_test_split(X_rest1, y_rest_g_peak_1, test_size = 1500, random_state=965)
X_calib, X_new, y_calib_g_peak, y_new_g_peak = train_test_split(X_rest2, y_rest_g_peak_2, test_size = 500, random_state=965)

# X_train, X_test, y_train_o_peak, y_test_o_peak = train_test_split(X, y_o_peak, test_size=0.2, random_state=965)
X_train, X_rest1, y_train_o_peak, y_rest_o_peak_1 = train_test_split(X,y_o_peak, test_size=2000, random_state=965)
X_test, X_rest2, y_test_o_peak, y_rest_o_peak_2 = train_test_split(X_rest1, y_rest_o_peak_1, test_size = 1500, random_state=965)
X_calib, X_new, y_calib_o_peak, y_new_o_peak = train_test_split(X_rest2, y_rest_o_peak_2, test_size = 500, random_state=965)

# X_train, X_test, y_train_w_cum, y_test_w_cum = train_test_split(X, y_w_cum, test_size=0.2, random_state=965)
X_train, X_rest1, y_train_w_cum, y_rest_w_cum_1 = train_test_split(X,y_w_cum, test_size=2000, random_state=965)
X_test, X_rest2, y_test_w_cum, y_rest_w_cum_2 = train_test_split(X_rest1, y_rest_w_cum_1, test_size = 1500, random_state=965)
X_calib, X_new, y_calib_w_cum, y_new_w_cum = train_test_split(X_rest2, y_rest_w_cum_2, test_size = 500, random_state=965)

# X_train, X_test, y_train_g_cum, y_test_g_cum = train_test_split(X, y_g_cum, test_size=0.2, random_state=965)
X_train, X_rest1, y_train_g_cum, y_rest_g_cum_1 = train_test_split(X,y_g_cum, test_size=2000, random_state=965)
X_test, X_rest2, y_test_g_cum, y_rest_g_cum_2 = train_test_split(X_rest1, y_rest_g_cum_1, test_size = 1500, random_state=965)
X_calib, X_new, y_calib_g_cum, y_new_g_cum = train_test_split(X_rest2, y_rest_g_cum_2, test_size = 500, random_state=965)

# X_train, X_test, y_train_o_cum, y_test_o_cum = train_test_split(X, y_o_cum, test_size=0.2, random_state=965)
X_train, X_rest1, y_train_o_cum, y_rest_o_cum_1 = train_test_split(X,y_o_cum, test_size=2000, random_state=965)
X_test, X_rest2, y_test_o_cum, y_rest_o_cum_2 = train_test_split(X_rest1, y_rest_o_cum_1, test_size = 1500, random_state=965)
X_calib, X_new, y_calib_o_cum, y_new_o_cum = train_test_split(X_rest2, y_rest_o_cum_2, test_size = 500, random_state=965)


## Boosted Tree Model

Scikit-learn reference:

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html#sklearn-ensemble-gradientboostingregressor

### Doing a GridSearchCV


In [ ]:
# # Define the parameter grid
# param_grid = {
#     'learning_rate': [0.01, 0.75, 0.1, 0.25],
#     'n_estimators': [300, 400, 500, 750],
#     'max_depth': [5, 7, 9, 11],
#     'alpha': [0.1, 0.5, 0.75, 0.999]
# }
# gb_mod_t = XGBRegressor(random_state=965)
# grid_search = GridSearchCV(estimator=gb_mod_t, param_grid=param_grid, cv = 2, scoring='r2')
# # Fit the grid search to your data


In [ ]:
# Grid Search, which is worth skipping

# grid_search.fit(X_train, y_train_w_3)

In [ ]:
# Get the best model and its parameters
# best_model = grid_search.best_estimator_
# best_params = grid_search.best_params_

# Print the best parameters and score
# print("Best parameters:", best_params)
# print("Best score:", grid_search.best_score_)

In [ ]:
# pd.DataFrame(grid_search.cv_results_)

### Doing a Much faster RandomSearchCV

In [ ]:
# Define distributions for hyperparameters
# from scipy.stats import uniform, randint
# param_dist = {
#     'learning_rate': uniform(0.05, 0.80),
#     'n_estimators': randint(300, 1000),
#     'max_depth': randint(5, 13),
#     'alpha': uniform(0.2, 0.8)
# }

In [ ]:
# # Specify the number of iterations for random search
# n_iter_search = 10

# # Create the RandomizedSearchCV object
# random_search = RandomizedSearchCV(estimator=gb_mod_t, param_distributions=param_dist, n_iter=n_iter_search, cv=5)

In [ ]:
# random_search.fit(X_train, y_train_w_3)

In [ ]:
# best_model = random_search.best_estimator_
# best_score = random_search.best_score_
# # 
# # Print the best parameters and score
# print("Best parameters:", best_params)
# print("Best score:", best_score)

In [ ]:
# pd.DataFrame(random_search.cv_results_)

### We will do Water First

In [ ]:
# gb_mod_0 = XGBRegressor(learning_rate=0.1, n_estimators= 300, max_depth = 7, random_state=965, alpha = 0.99)
# gb_mod_0.fit(X_train, y_train_w_3)
# print("XG Boost (default parameters) Train R2: ", gb_mod_0.score(X_train, y_train_w_3))
# print("XG Boost (default parameters) Test R2: ", gb_mod_0.score(X_test, y_test_w_3))

In [ ]:
# gb_mod_1 = XGBRegressor(learning_rate=0.01, n_estimators= 300, max_depth = 7, random_state=965, alpha = 0.99)
# gb_mod_1.fit(X_train, y_train_w_3)
# print("XG Boost (default parameters) Train R2: ", gb_mod_1.score(X_train, y_train_w_3))
# print("XG Boost (default parameters) Test R2: ", gb_mod_1.score(X_test, y_test_w_3))

In [ ]:
# gb_mod_2 = XGBRegressor(learning_rate=1, n_estimators= 300, max_depth = 7, random_state=965, alpha = 0.99)
# gb_mod_2.fit(X_train, y_train_w_3)
# print("XG Boost (default parameters) Train R2: ", gb_mod_2.score(X_train, y_train_w_3))
# print("XG Boost (default parameters) Test R2: ", gb_mod_2.score(X_test, y_test_w_3))

In [ ]:
# gb_mod_3 = XGBRegressor(learning_rate=0.1, n_estimators= 300, max_depth = 9, random_state=965, alpha = 0.99)
# gb_mod_3.fit(X_train, y_train_w_3)
# print("XG Boost (default parameters) Train R2: ", gb_mod_3.score(X_train, y_train_w_3))
# print("XG Boost (default parameters) Test R2: ", gb_mod_3.score(X_test, y_test_w_3))

In [ ]:
# gb_mod_4 = XGBRegressor(learning_rate=0.075, n_estimators= 500, max_depth = 11, random_state=965, alpha = 0.99)
# gb_mod_4.fit(X_train, y_train_w_3)
# print("XG Boost (default parameters) Train R2: ", gb_mod_4.score(X_train, y_train_w_3))
# print("XG Boost (default parameters) Test R2: ", gb_mod_4.score(X_test, y_test_w_3))

In [ ]:
# gb_mod_5 = XGBRegressor(learning_rate=0.075, n_estimators= 500, max_depth = 9, random_state=965, alpha = 0.99)
# gb_mod_5.fit(X_train, y_train_w_3)
# print("XG Boost (default parameters) Train R2: ", gb_mod_5.score(X_train, y_train_w_3))
# print("XG Boost (default parameters) Test R2: ", gb_mod_5.score(X_test, y_test_w_3))

In [ ]:
# gb_mod_6 = XGBRegressor(learning_rate=0.075, n_estimators= 500, max_depth = 9, random_state=965, alpha = 0.99)
# gb_mod_6.fit(X_train, y_train_w_3)
# print("XG Boost (default parameters) Train R2: ", gb_mod_6.score(X_train, y_train_w_3))
# print("XG Boost (default parameters) Test R2: ", gb_mod_6.score(X_test, y_test_w_3))

### Fucking Oil Man

In [ ]:
# gb_mod_7 = XGBRegressor(learning_rate=0.075, n_estimators= 400, max_depth = 7, random_state=965, alpha = 0.5)
# gb_mod_7.fit(X_train, y_train_o_3)
# print("XG Boost (default parameters) Train R2: ", gb_mod_7.score(X_train, y_train_o_3))
# print("XG Boost (default parameters) Test R2: ", gb_mod_7.score(X_test, y_test_o_3))

In [ ]:
# gb_mod_8 = XGBRegressor(learning_rate=0.075, n_estimators= 500, max_depth = 10, random_state=965, alpha = 0.5)
# gb_mod_8.fit(X_train, y_train_o_3)
# print("XG Boost (default parameters) Train R2: ", gb_mod_8.score(X_train, y_train_o_3))
# print("XG Boost (default parameters) Test R2: ", gb_mod_8.score(X_test, y_test_o_3))

In [ ]:
# gb_mod_9 = XGBRegressor(learning_rate=0.1, n_estimators= 700, max_depth = 10, random_state=965, alpha = 0.5)
# gb_mod_9.fit(X_train, y_train_o_3)
# print("XG Boost (default parameters) Train R2: ", gb_mod_9.score(X_train, y_train_o_3))
# print("XG Boost (default parameters) Test R2: ", gb_mod_9.score(X_test, y_test_o_3))

In [ ]:
# gb_mod_10 = XGBRegressor(learning_rate=0.1, n_estimators= 500, max_depth = 10, random_state=965, alpha = 0.5)
# gb_mod_10.fit(X_train, y_train_o_3)
# print("XG Boost (default parameters) Train R2: ", gb_mod_10.score(X_train, y_train_o_3))
# print("XG Boost (default parameters) Test R2: ", gb_mod_10.score(X_test, y_test_o_3))

In [16]:
# One model to run test on 
gb_mod_11 = XGBRegressor(learning_rate=0.035, n_estimators= 450, max_depth = 10, random_state=965, alpha = 0.6)
gb_mod_11.fit(X_train, y_train_o_3)
print("XG Boost (default parameters) Train R2: ", gb_mod_11.score(X_train, y_train_o_3))
print("XG Boost (default parameters) Test R2: ", gb_mod_11.score(X_test, y_test_o_3))

XG Boost (default parameters) Train R2:  0.9492561131482483
XG Boost (default parameters) Test R2:  0.3695324445064202


In [ ]:
# Create an empty dictionary to store XGBRegressor instances
gb_mod_10 = {}

# Loop through the range and create/update dictionary entries
for num in range(10):
    var_name = f'gb_mod_10_{num}'  # Construct variable name dynamically
    gb_mod_10[var_name] = XGBRegressor(learning_rate=0.06 + (0.005 * num), n_estimators=500, max_depth=10, random_state=965, alpha=0.5)
    gb_mod_10[var_name].fit(X_train, y_train_o_3)
    print("XG Boost (learning rate = ",0.06 + (0.005 * num),") Train R2: ", gb_mod_10[var_name].score(X_train, y_train_o_3))
    print("XG Boost (learning rate = ",0.06 + (0.005 * num),") Test R2: ", gb_mod_10[var_name].score(X_test, y_test_o_3))

In [ ]:
# Create an empty dictionary to store XGBRegressor instances
gb_mod_10 = {}

# Loop through the range and create/update dictionary entries
for num in range(6):
    var_name = f'gb_mod_10_{num}'  # Construct variable name dynamically
    gb_mod_10[var_name] = XGBRegressor(learning_rate=0.03 + (0.005 * num), n_estimators=500, max_depth=10, random_state=965, alpha=0.5)
    gb_mod_10[var_name].fit(X_train, y_train_o_3)
    print("XG Boost (learning rate = ",0.03 + (0.005 * num),") Train R2: ", gb_mod_10[var_name].score(X_train, y_train_o_3))
    print("XG Boost (learning rate = ",0.03 + (0.005 * num),") Test R2: ", gb_mod_10[var_name].score(X_test, y_test_o_3))

In [ ]:
lr = 0.035 # just optimized for this
md = 10
est = 200
alp = 0.5

# Create an empty dictionary to store XGBRegressor instances
gb_mod_10 = {}

# Loop through the range and create/update dictionary entries
for num in range(10):
    var_name = f'gb_mod_10_{num}'  # Construct variable name dynamically
    gb_mod_10[var_name] = XGBRegressor(learning_rate=lr, n_estimators=est + (50 * num), max_depth=md, random_state=965, alpha=alp)
    gb_mod_10[var_name].fit(X_train, y_train_o_3)
    print("XG Boost ( estimators = ",est + (50 * num),") Train R2: ", gb_mod_10[var_name].score(X_train, y_train_o_3))
    print("XG Boost ( estimators = ",est + (50 * num),") Test R2: ", gb_mod_10[var_name].score(X_test, y_test_o_3))

In [ ]:
lr = 0.035  #optimized
md = 10     #optiimized
est = 450   #optimized
alp = 0.0
#lam = 1

# Create an empty dictionary to store XGBRegressor instances
gb_mod_10 = {}

# Loop through the range and create/update dictionary entries
for num in range(6):
    var_name = f'gb_mod_10_{num}'  # Construct variable name dynamically
    gb_mod_10[var_name] = XGBRegressor(learning_rate=lr, n_estimators=est, max_depth=md, random_state=965, alpha=alp + (0.2*num))
    gb_mod_10[var_name].fit(X_train, y_train_o_3)
    print("XG Boost ( alpha = ", alp + (0.2*num),") Train R2: ", gb_mod_10[var_name].score(X_train, y_train_o_3))
    print("XG Boost ( alpha = ", alp + (0.2*num),") Test R2: ", gb_mod_10[var_name].score(X_test, y_test_o_3))

In [ ]:
lr = 0.035  #optimized
md = 10     #optiimized
est = 450   #optimized
alp = 0.6   #optimized
lam = 0.25

# Create an empty dictionary to store XGBRegressor instances
gb_mod_10 = {}

# Loop through the range and create/update dictionary entries
for num in range(8):
    var_name = f'gb_mod_10_{num}'  # Construct variable name dynamically
    gb_mod_10[var_name] = XGBRegressor(learning_rate=lr, n_estimators=est, max_depth=md, random_state=965, alpha=alp, reg_lambda = lam + (0.25*num))
    gb_mod_10[var_name].fit(X_train, y_train_o_3)
    print("XG Boost ( lambda = ", lam + (0.25*num),") Train R2: ", gb_mod_10[var_name].score(X_train, y_train_o_3))
    print("XG Boost ( lambda = ", lam + (0.25*num),") Test R2: ", gb_mod_10[var_name].score(X_test, y_test_o_3))

In [ ]:
lr = 0.035  #optimized
md = 10     #optiimized
est = 450   #optimized
alp = 0.6   #optimized
lam = 1

# Create an empty dictionary to store XGBRegressor instances
gb_mod_10 = {}

# Loop through the range and create/update dictionary entries
for num in range(1):
    var_name = f'gb_mod_10_{num}'  # Construct variable name dynamically
    gb_mod_10[var_name] = XGBRegressor(learning_rate=lr, n_estimators=est, max_depth=md, random_state=965, alpha=alp)
    gb_mod_10[var_name].fit(X_train, y_train_o_3)
    print("XG Boost ( lambda = ", lam ,") Train R2: ", gb_mod_10[var_name].score(X_train, y_train_o_3))
    print("XG Boost ( lambda = ", lam ,") Test R2: ", gb_mod_10[var_name].score(X_test, y_test_o_3))

In [ ]:
X_train.head()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
y_pred = gb_mod_11.predict(X_test)
y_test = y_test_o_3

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {round(mae,2)}")
print(f"Mean Squared Error (MSE): {round(mse,2)}")
print(f"Root Mean Squared Error (RMSE): {round(rmse,2)}")
print(f"R-squared (R²): {round(r2,6)}")

## We have the optimized values below. Now we need to create a model for all time periods and all elements

In [54]:
# Great this works

list_of_train_columns=[
y_train_o_3,
y_train_o_6,
y_train_o_9,
y_train_o_12,
y_train_o_cum,
y_train_o_peak,
y_train_w_3,
y_train_w_6,
y_train_w_9,
y_train_w_12,
y_train_w_cum,
y_train_w_peak,
y_train_g_3,
y_train_g_6,
y_train_g_9,
y_train_g_12,
y_train_g_cum,
y_train_g_peak
]

list_of_test_columns = [
y_test_o_3,
y_test_o_6,
y_test_o_9,
y_test_o_12,
y_test_o_cum,
y_test_o_peak,
y_test_w_3,
y_test_w_6,
y_test_w_9,
y_test_w_12,
y_test_w_cum,
y_test_w_peak,
y_test_g_3,
y_test_g_6,
y_test_g_9,
y_test_g_12,
y_test_g_cum,
y_test_g_peak
]

list_of_y_calib_columns = [
y_calib_o_3,
y_calib_o_6,
y_calib_o_9,
y_calib_o_12,
y_calib_o_cum,
y_calib_o_peak,
y_calib_w_3,
y_calib_w_6,
y_calib_w_9,
y_calib_w_12,
y_calib_w_cum,
y_calib_w_peak,
y_calib_g_3,
y_calib_g_6,
y_calib_g_9,
y_calib_g_12,
y_calib_g_cum,
y_calib_g_peak    
]

list_of_y_new_columns = [
y_new_o_3,
y_new_o_6,
y_new_o_9,
y_new_o_12,
y_new_o_cum,
y_new_o_peak,
y_new_w_3,
y_new_w_6,
y_new_w_9,
y_new_w_12,
y_new_w_cum,
y_new_w_peak,
y_new_g_3,
y_new_g_6,
y_new_g_9,
y_new_g_12,
y_new_g_cum,
y_new_g_peak 
]

In [55]:
display_list_columns=[
'o_3',
'o_6',
'o_9',
'o_12',
'o_cum',
'o_peak',
'w_3',
'w_6',
'w_9',
'w_12',
'w_cum',
'w_peak',
'g_3',
'g_6',
'g_9',
'g_12',
'g_cum',
'g_peak'
]

In [56]:
# Create a list of tuples by zipping train_list and test_list
data_tuples = []
for i in range(min(len(list_of_train_columns), len(list_of_test_columns))):
    data_tuples.append((list_of_train_columns[i], list_of_test_columns[i]))

In [97]:
# these them models
lr = 0.035  #optimized
md = 10     #optiimized
est = 450   #optimized
alp = 0.6   #optimized
lam = 1     #optimized

# Create an empty dictionary to store XGBRegressor instances
boosted_models_list = {}
y_pred = {}
# Loop through the range and create/update dictionary entries
for i in range(min(len(list_of_train_columns), len(list_of_test_columns))):
    var_name = f'xgb_mod_{display_list_columns[i]}'  # Construct variable name dynamically
    boosted_models_list[var_name] = XGBRegressor(learning_rate=lr, n_estimators=est, max_depth=md, random_state=965, alpha=alp)
    train_ref = data_tuples[i][0]
    test_ref = data_tuples[i][1]
    boosted_models_list[var_name].fit(X_train, train_ref)
    y_pred[i] = boosted_models_list[var_name].predict(X_test)
    print("XG Boost ( train set = ", display_list_columns[i],") Train R2: ", boosted_models_list[var_name].score(X_train, train_ref))
    print("XG Boost ( test set = ", display_list_columns[i],") Test R2: ", boosted_models_list[var_name].score(X_test, test_ref))

XG Boost ( train set =  o_3 ) Train R2:  0.9470239506732545
XG Boost ( test set =  o_3 ) Test R2:  0.3897839514401895
XG Boost ( train set =  o_6 ) Train R2:  0.9567971259485112
XG Boost ( test set =  o_6 ) Test R2:  0.44254937146942275
XG Boost ( train set =  o_9 ) Train R2:  0.9597185762764434
XG Boost ( test set =  o_9 ) Test R2:  0.4605571708281735
XG Boost ( train set =  o_12 ) Train R2:  0.9607609597324354
XG Boost ( test set =  o_12 ) Test R2:  0.4808869868618745
XG Boost ( train set =  o_cum ) Train R2:  0.9569279803767942
XG Boost ( test set =  o_cum ) Test R2:  0.39443193255661024
XG Boost ( train set =  o_peak ) Train R2:  0.9555697360528334
XG Boost ( test set =  o_peak ) Test R2:  0.31668587018285155
XG Boost ( train set =  w_3 ) Train R2:  0.9511878339142115
XG Boost ( test set =  w_3 ) Test R2:  0.40253536746506857
XG Boost ( train set =  w_6 ) Train R2:  0.9545163278407647
XG Boost ( test set =  w_6 ) Test R2:  0.42533082561123314
XG Boost ( train set =  w_9 ) Train R2:

In [58]:
print("Shape of X_train:", X_train.shape)
print("Shape of train_ref:", train_ref.shape)

Shape of X_train: (11940, 21)
Shape of train_ref: (11940,)


In [42]:
list_of_y_calib_columns[2]

6786      60788.0
12588     48511.0
9330     260082.0
6717     102848.0
9649      79250.0
9321     166160.0
7907     141888.0
11962    240591.0
9389     128676.0
14810    223118.0
6879      56449.0
2537      76078.0
6224     160636.0
5815     125729.0
1209      78839.0
13564    134560.0
16019    103498.0
14671    340335.0
8409     132689.0
14859    181566.0
4324      82632.0
2562      88595.0
13100    192318.0
15748         0.0
15085      8675.0
14857    120602.0
14648    187369.0
12946     92657.0
6089      70896.0
14709    211209.0
3953      65191.0
13127    193127.0
14437    126051.0
7459     162556.0
8466     118648.0
2362      93783.0
7964     123495.0
9791     172244.0
12845    136073.0
574       25937.0
5138     183350.0
3344     111265.0
12131    107756.0
2211      30175.0
4642      93787.0
2838      78518.0
15235    106141.0
8045     207377.0
6585     176699.0
13579    103602.0
13124    216691.0
2408      84227.0
1104      34602.0
3242      31139.0
8569     192452.0
3962      

In [51]:
print(type(boosted_models_list))

<class 'dict'>


In [66]:
list(boosted_models_list.items())[1][0]

'xgb_mod_o_6'

In [69]:
pip install mapie

     -------------------------------------- 141.1/141.1 kB 1.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [70]:
from mapie.regression import MapieRegressor

In [ ]:
# Loop through to create y_pred

for mod in 
y_pred = gb_mod_3.predict(X_test)
mae=mean_absolute_error(y_test_w_3,y_pred)
print(round(mae,2))

In [115]:
# Need to create the mapie regressor from the gb_mods
# This somehow then needs to look through associated x and ys on the fit
n=0
mapie_reg_list = {}
for guy in display_list_columns:
    var_name = f'mapie_reg_{guy}'
    mapie_reg_list[var_name] = MapieRegressor(estimator=list(boosted_models_list.items())[n][1],cv="prefit").fit(X_calib, list_of_y_calib_columns[n])
    # y_pred[n], y_pis = mapie_reg_list[var_name].predict(X_new,alpha=1/3)
    n = n+1

In [105]:
type(boosted_models_list)

dict

In [106]:
first_key = next(iter(boosted_models_list))

In [108]:
boosted_models_list[first_key]

XGBRegressor(alpha=0.6, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.035, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=450, n_jobs=None,
             num_parallel_tree=None, ...)

In [109]:
for key, value in boosted_models_list.items():
    # Do something with key and value
    print(key, value)  # For example, print the key and value

xgb_mod_o_3 XGBRegressor(alpha=0.6, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.035, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=450, n_jobs=None,
             num_parallel_tree=None, ...)
xgb_mod_o_6 XGBRegressor(alpha=0.6, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical

In [114]:
list(boosted_models_list.items())[0][1]

XGBRegressor(alpha=0.6, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.035, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=450, n_jobs=None,
             num_parallel_tree=None, ...)

In [120]:
list(mapie_reg_list.items())[3][1]

MapieRegressor(cv='prefit',
               estimator=XGBRegressor(alpha=0.6, base_score=None, booster=None,
                                      callbacks=None, colsample_bylevel=None,
                                      colsample_bynode=None,
                                      colsample_bytree=None, device=None,
                                      early_stopping_rounds=None,
                                      enable_categorical=False,
                                      eval_metric=None, feature_types=None,
                                      gamma=None, grow_policy=None,
                                      importance_type=None,
                                      interaction_constraints=None,
                                      learning_rate=0.035, max_bin=None,
                                      max_cat_threshold=None,
                                      max_cat_to_onehot=None,
                                      max_delta_step=None, max_depth=10,
       

In [121]:
list(mapie_reg_list.items())

[('mapie_reg_o_3',
  MapieRegressor(cv='prefit',
                 estimator=XGBRegressor(alpha=0.6, base_score=None, booster=None,
                                        callbacks=None, colsample_bylevel=None,
                                        colsample_bynode=None,
                                        colsample_bytree=None, device=None,
                                        early_stopping_rounds=None,
                                        enable_categorical=False,
                                        eval_metric=None, feature_types=None,
                                        gamma=None, grow_policy=None,
                                        importance_type=None,
                                        interaction_constraints=None,
                                        learning_rate=0.035, max_bin=None,
                                        max_cat_threshold=None,
                                        max_cat_to_onehot=None,
                                 

In [122]:
# Need to loop through the pis as well
n = 0
y_pis = {}
for guy in display_list_columns:
    y_pred[n],y_pis[n] = list(mapie_reg_list.items())[n][1].predict(X_new,alpha=1/3)
    n = n+1

In [128]:
y_pred[0][0]

22857.021

In [132]:
float(y_pis[0][0][0])

6204.58984375

In [133]:
for i in range(5):
    print(f"Well {i+1}")
    #print(X_new_o_3.iloc[i])  # Print the features of the i-th well
    print(f"Predicted water: {y_pred[0][i]:.2f}")  # Print the predicted oil
    
    #interval = y_pis[i].flatten()  # Flatten the interval if it's not already a 1D array
    print(f"67% interval: [{float(y_pis[0][0][0]):.2f};{float(y_pis[0][0][1]):.2f}]")  # Print the 67% prediction interval
    print("---") # Separate the Well data

Well 1
Predicted water: 22857.02
67% interval: [6204.59;39509.45]
---
Well 2
Predicted water: 42953.53
67% interval: [6204.59;39509.45]
---
Well 3
Predicted water: 37368.96
67% interval: [6204.59;39509.45]
---
Well 4
Predicted water: 28244.55
67% interval: [6204.59;39509.45]
---
Well 5
Predicted water: 28721.84
67% interval: [6204.59;39509.45]
---


## Let's make some fucking charts

In [ ]:
feature_names = X_train.columns
# Extract feature importances from the model
importances = gb_mod_11.feature_importances_
# Sort features and importances in descending order of importance
sorted_idx = importances.argsort()[::-1]
sorted_names = [feature_names[i] for i in sorted_idx][::-1]
sorted_importances = importances[sorted_idx][::-1]

# Create the bar plot
plt.figure(figsize=(10, 6))  # Adjust figure size as needed
plt.barh(sorted_names, sorted_importances)
plt.xlabel('Feature Importance')
plt.ylabel('Feature Name')
plt.title('Feature Importance for Gradient Boosting Model')
plt.xticks(rotation=45, ha='right', fontsize = 8)  # Rotate feature names for better readability
plt.yticks(fontsize = 8)
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.tree import plot_tree

# Choose the tree index to visualize (between 0 and number of trees - 1)
tree_index = 4  # Change this to the desired tree index

# Extract the tree object from the model
tree = gb_mod_5.estimators_[tree_index]

In [ ]:
from sklearn.tree import export_graphviz
export_graphviz(
        gb_mod_5,
        out_file="tree.dot",
        feature_names=X_train.columns,
        impurity=False,
        rounded=True,
        filled=True
    )
Source.from_file("tree.dot")

In [ ]:
dff.describe()

In [ ]:
df.head()

In [ ]:
# Sample data (modify with your actual data)
var1 = dff['TrueVerticalDepth_FT']
var2 = dff['MeasuredDepth_FT']

# Create the plot
plt.hist(var1, bins='auto', alpha=0.5, label='Vertical Depth')
plt.hist(var2, bins='auto', alpha=0.5, label='Full Measured Length')
plt.xlabel('Feet')
plt.ylabel('Frequency')
plt.title('Frequency Distribution of Well Depth')
plt.legend()
plt.grid(False)
plt.show()

In [ ]:
# Sample data (modify with your actual data)
var1 = dff['CumOil_BBL']

# Create the plot
plt.hist(var1, bins='auto', alpha=0.5)
plt.xlabel('Barrels of Oil')
plt.ylabel('Frequency')
plt.title('Frequency Distribution of Oil Production in Barrels')
plt.legend()
plt.grid(False)
plt.show()

In [ ]:
# Sample data (modify with your actual data)
var1 = dff['ProductionMonthsCount']

# Create the plot
plt.hist(var1, bins='auto', alpha=0.5)
plt.xlabel('Number of Months')
plt.ylabel('Frequency')
plt.title('Frequency Distribution of Production Timeline per Well')
plt.legend()
plt.grid(False)
plt.show()

In [ ]:
# Create the bar plot
# new imports
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
scaler = StandardScaler()
scaler.fit(X_train)
std_x_train = X_train.copy()
std_x_test = X_test.copy()

std_train_array = scaler.transform(std_x_train)
std_test_array = scaler.transform(std_x_test)

std_x_train[:] = std_train_array
std_x_test[:] = std_test_array

# Apply PCA
pca = PCA(n_components=len(X_train.columns))
pca.fit(std_x_train[:])


# Example data: Explained variance ratio for each principal component
explained_variance_ratio = np.array(pca.explained_variance_ratio_)

# Cumulative explained variance
cumulative_explained_variance = pca.explained_variance_ratio_.cumsum()

# Number of components
components = range(1, len(explained_variance_ratio) + 1)

# Creating the plot
plt.figure(figsize=(10, 6))
plt.bar(components, explained_variance_ratio, alpha=0.5, label='Individual explained variance')
plt.plot(components, cumulative_explained_variance, marker='o', linestyle='-', color='r', label='Cumulative explained variance')

plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('PCA Explained Variance')
plt.xticks(components, X_train.columns[:pca.n_components_], rotation=45, fontsize = 8, ha='right')
plt.legend(loc='best')

plt.show()
